In [ ]:
# Telegram Data Collection for Ukraine-Russia Conflict Analysis
# This notebook scrapes Telegram channels from Feb 24, 2022 to June 6, 2025

# Telegram Data Collection for Ukraine-Russia Conflict Analysis
 
This notebook implements a comprehensive Telegram scraping system for collecting grassroots discourse about the Ukraine-Russia conflict from February 24, 2022 (invasion date) to present.

## Key Features:
- Balanced collection from pro-Ukrainian and pro-Russian channels
- Rate limiting and anti-ban measures
- Multilingual text processing (Russian, Ukrainian, English)
- Sentiment analysis preparation
- SQLite storage with comprehensive metadata

## 1. Setup and Dependencies

In [ ]:
import asyncio
import sqlite3
import json
import pandas as pd
import re
import time
import random
from datetime import datetime, timezone, timedelta
from telethon import TelegramClient
from telethon.errors import FloodWaitError, ChannelPrivateError
from langdetect import detect
import logging

In [ ]:
# Install tqdm if needed
import subprocess
import sys

try:
    from tqdm import tqdm
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tqdm"])
    from tqdm import tqdm

In [ ]:
# --- repo bootstrap ---------------------------------------------------------
from pathlib import Path
from dotenv import load_dotenv
import os, sys

def repo_root(start: Path) -> Path:
    cur = start.resolve()
    while cur != cur.parent:
        if (cur / ".env").exists() or (cur / ".git").exists():
            return cur
        cur = cur.parent
    raise RuntimeError("repo root not found")

ROOT = repo_root(Path.cwd())
load_dotenv(ROOT / ".env")             # loads secrets
sys.path.append(str(ROOT / "src"))     # optional helpers

DATA_DIR = ROOT / "data"
OUT_DIR  = ROOT / "outputs"
FIG_DIR  = OUT_DIR / "figs"; FIG_DIR.mkdir(exist_ok=True)

print("Repo root:", ROOT)

### Hard Coded Timeline Events

In [ ]:
# Load the timeline events
timeline_events = [
    {"date":"2022-02-24","event":"Russia launches full-scale invasion of Ukraine","label":"Invasion","major":True},
    {"date":"2022-04-02","event":"Bucha massacre discovered","label":"Bucha","major":True},
    {"date":"2022-05-20","event":"Mariupol falls","label":"Mariupol Falls","major":True},
    {"date":"2022-09-11","event":"Ukrainian counteroffensive liberates Kharkiv","label":"Kharkiv Liberation","major":True},
    {"date":"2022-11-11","event":"Ukraine liberates Kherson city","label":"Kherson Free","major":True},
    {"date":"2023-05-20","event":"Bakhmut falls","label":"Bakhmut Falls","major":True},
    {"date":"2023-06-06","event":"Kakhovka Dam destroyed","label":"Dam Destroyed","major":True},
    {"date":"2024-02-17","event":"Russia captures Avdiivka","label":"Avdiivka Falls","major":True},
    {"date":"2024-08-06","event":"Ukraine launches Kursk incursion","label":"Kursk","major":True},
    {"date":"2024-11-05","event":"Trump wins US election","label":"Trump Elected","major":True},
    {"date":"2025-01-20","event":"Trump inaugurated","label":"Trump Policy","major":True},
    {"date":"2025-02-28","event":"Trump-Zelensky Oval Office confrontation","label":"US-Ukraine Rift","major":True},
    {"date":"2025-05-17","event":"Istanbul talks","label":"Istanbul","major":True},
    {"date":"2025-05-19","event":"Trump announces 'immediate' ceasefire talks","label":"Peace Claims","major":True},
    {"date":"2025-05-29","event":"Ukrainian drone 'Trojan Horse' infiltrates Russian base","label":"Drone Trojan Horse","major":True},
    {"date":"2025-06-02","event":"Russia intensifies Chasiv Yar assault","label":"Chasiv Yar","major":True},
    {"date":"2025-06-04","event":"NATO announces expanded weapons package","label":"NATO Aid","major":True}
]

# Convert to datetime objects
for event in timeline_events:
    event['datetime'] = datetime.strptime(event['date'], '%Y-%m-%d').replace(tzinfo=timezone.utc)

### API & Channel Setup

In [ ]:
# API Configuration (you'll need to get these from https://my.telegram.org)
API_ID = os.getenv('TELEGRAM_API_ID', 'YOUR_API_ID')  # Store in .env file
API_HASH = os.getenv('TELEGRAM_API_HASH', 'YOUR_API_HASH')  # Store in .env file
PHONE = os.getenv('TELEGRAM_PHONE', '+YOUR_PHONE_NUMBER')  # Store in .env file

# Setup logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(OUT_DIR / 'telegram_scraper.log'),
        logging.StreamHandler()
    ]
)

# Channel lists for balanced collection
CHANNELS = {
    'pro_ukrainian_grassroots': [
        '@ssternenko',  # Serhii Sternenko (activist/volunteer)
        '@operativnoZSU',  # AFU Operational
        '@voynareal',  # War Real
        '@DeepStateUA',  # DeepState UA (mapping)
        '@horevica',  # Military blogger
        '@ukr_sof',  # UA Special Forces
        '@aerobomber',  # UA Drone operations
        '@combat_ftg'  # Combat footage
    ],
    'pro_russian_grassroots': [
        '@rybar',  # Rybar (major military blogger)
        '@starshe_eddy',  # Starshe Eddy (war correspondent)
        '@wargonzo',  # WarGonzo
        '@grey_zone',  # Grey Zone (Wagner)
        '@vysokygovorit',  # Military analyst
        '@milinfolive',  # Military Informant
        '@voenkorKotenok',  # War correspondent
        '@romanov_92'  # Military blogger
    ],
    'neutral_independent': [
        '@astra',  # Independent Russian media
        '@CITeam_en',  # Conflict Intelligence Team
        '@bellingcat',  # Investigative journalism
        '@GirkinGirkin'  # Igor Girkin updates
    ],
    'official_comparison': [
        '@V_Zelenskiy_official',  # President Zelenskyy
        '@spravdi',  # Ukraine Gov
        '@mod_russia',  # Russian MoD
        '@MID_RF'  # Russian MFA
    ]
}

# Test channels for initial run
TEST_CHANNELS = {
    'test_ukrainian': ['@operativnoZSU'],  # High activity Ukrainian channel
    'test_russian': ['@rybar'],  # High activity Russian channel
    'test_neutral': ['@astra']  # Independent channel
}

print(f"✅ Configuration loaded")
print(f"   Total channels configured: {sum(len(ch) for ch in CHANNELS.values())}")
print(f"   Test channels selected: {sum(len(ch) for ch in TEST_CHANNELS.values())}")

### Generate Sampling Schedule

In [ ]:
def generate_sampling_schedule(start_date, end_date, events, baseline_interval=3, event_window=14):
    """
    Generate dates to scrape based on:
    - Baseline: every N days throughout the conflict
    - Intensive: daily for ±event_window days around major events
    """
    schedule = set()
    
    # 1. Baseline sampling (every 3rd day)
    current = start_date
    while current <= end_date:
        schedule.add(current.date())
        current += timedelta(days=baseline_interval)
    
    # 2. Intensive sampling around major events
    for event in events:
        if event.get('major', False):
            event_date = event['datetime']
            # Sample daily for ±14 days around event
            for offset in range(-event_window, event_window + 1):
                sample_date = event_date + timedelta(days=offset)
                if start_date <= sample_date <= end_date:
                    schedule.add(sample_date.date())
    
    # Convert back to sorted list
    schedule_list = sorted(list(schedule))
    
    return schedule_list

# Generate the schedule
START_DATE = datetime(2022, 2, 24, tzinfo=timezone.utc)
END_DATE = datetime(2025, 6, 6, tzinfo=timezone.utc)

sampling_dates = generate_sampling_schedule(START_DATE, END_DATE, timeline_events)

print(f"📅 Sampling Strategy Summary:")
print(f"   Total days in conflict: {(END_DATE - START_DATE).days}")
print(f"   Days to scrape: {len(sampling_dates)} ({len(sampling_dates)/(END_DATE - START_DATE).days*100:.1f}%)")
print(f"   Time saved: {100 - len(sampling_dates)/(END_DATE - START_DATE).days*100:.1f}%")

### Visualize Sampling Density

In [ ]:
# Create a visualization of sampling density
import matplotlib.pyplot as plt

# Count samples per month
monthly_samples = {}
for date in sampling_dates:
    month_key = f"{date.year}-{date.month:02d}"
    monthly_samples[month_key] = monthly_samples.get(month_key, 0) + 1

# Plot
months = sorted(monthly_samples.keys())
counts = [monthly_samples[m] for m in months]

plt.figure(figsize=(15, 6))
plt.bar(range(len(months)), counts)
plt.xticks(range(0, len(months), 3), months[::3], rotation=45)
plt.title('Telegram Sampling Density by Month')
plt.xlabel('Month')
plt.ylabel('Days to Scrape')

# Mark major events
for event in timeline_events:
    if event.get('major'):
        event_month = f"{event['datetime'].year}-{event['datetime'].month:02d}"
        if event_month in months:
            idx = months.index(event_month)
            plt.annotate(event['label'], xy=(idx, counts[idx]), 
                        xytext=(idx, counts[idx] + 2), rotation=90, fontsize=8)

plt.tight_layout()
plt.show()


### Priority Channel Sampling

In [ ]:
def estimate_channel_importance(channel_username, category):
    """
    Assign priority scores to channels for selective scraping
    Higher score = more important to scrape completely
    """
    scores = {
        # High priority military channels
        '@rybar': 10,  # Major Russian military blogger
        '@operativnoZSU': 10,  # Ukrainian military official
        '@starshe_eddy': 9,  # Detailed Russian reports
        '@_3_brigade': 9,  # Elite Ukrainian unit
        
        # Medium priority
        '@wargonzo': 7,
        '@grey_zone': 7,
        '@astra': 8,  # Independent, anti-Kremlin
        
        # Lower priority (sample more sparsely)
        '@brussinf': 5,
        '@combat_ftg': 5,
    }
    
    # Default scores by category
    category_defaults = {
        'pro_ukrainian': 7,
        'pro_russian': 7,
        'neutral': 8,
        'official': 6  # Lower priority, we have Truth Social for officials
    }
    
    return scores.get(channel_username, category_defaults.get(category, 5))

### Optimized Scraping Function

In [ ]:
async def scrape_with_sampling(client, channel_username, category, conn, sampling_dates):
    """
    Scrape channel but only for dates in our sampling schedule
    This dramatically reduces the amount of data to collect
    """
    cursor = conn.cursor()
    messages_collected = 0
    channel_priority = estimate_channel_importance(channel_username, category)
    
    try:
        channel = await client.get_entity(channel_username)
        
        # Store channel metadata
        cursor.execute('''
            INSERT OR REPLACE INTO channels 
            (username, title, category, priority_score, last_scraped)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            channel_username,
            getattr(channel, 'title', channel_username),
            category,
            channel_priority,
            datetime.now()
        ))
        
        # Convert sampling_dates to set for faster lookup
        sample_date_set = set(sampling_dates)
        
        # For high-priority channels during major events, sample more densely
        async for message in client.iter_messages(channel, limit=None):
            # Check if message date is in our sampling schedule
            message_date = message.date.date() if message.date else None
            
            if message_date not in sample_date_set:
                continue
                
            # For very high priority channels (score >= 9), also collect adjacent days
            if channel_priority >= 9:
                # Check if within 1 day of a scheduled date
                nearby_dates = [message_date + timedelta(days=i) for i in [-1, 0, 1]]
                if not any(d in sample_date_set for d in nearby_dates):
                    continue
            
            # Rate limiting - less aggressive for sampled approach
            if messages_collected % 50 == 0:
                await asyncio.sleep(0.5)
            
            # Store message (same as before)
            if message.text:
                cursor.execute('''
                    INSERT OR IGNORE INTO messages 
                    (channel_username, channel_category, message_id, date, message_text, ...)
                    VALUES (?, ?, ?, ?, ?, ...)
                ''', (...))  # Same as original
                
                messages_collected += 1
                
                if messages_collected % 100 == 0:
                    conn.commit()
                    print(f"   {channel_username}: {messages_collected} messages")
                    
    except Exception as e:
        print(f"Error with {channel_username}: {e}")
    
    conn.commit()
    return messages_collected

### Execution Strategy

In [ ]:
async def main_event_driven():
    """
    Main execution function with event-driven sampling
    """
    
    # Initialize client
    client = TelegramClient('ukraine_russia_event_sampling', API_ID, API_HASH)
    await client.start(PHONE)
    
    # Connect to database
    conn = sqlite3.connect('telegram_sampled_data.db')
    
    # Create tables (include priority_score field)
    create_database_with_priority(conn)
    
    total_messages = 0
    channel_stats = {}
    
    # Process channels by priority
    all_channels = []
    for category, channel_list in CHANNELS.items():
        for channel in channel_list:
            priority = estimate_channel_importance(channel, category)
            all_channels.append((priority, channel, category))
    
    # Sort by priority (highest first)
    all_channels.sort(reverse=True, key=lambda x: x[0])
    
    print(f"\n📊 Processing {len(all_channels)} channels by priority")
    
    for priority, channel, category in all_channels:
        print(f"\n🔍 Scraping {channel} (priority: {priority}, category: {category})")
        
        messages = await scrape_with_sampling(
            client, channel, category, conn, sampling_dates
        )
        
        channel_stats[channel] = messages
        total_messages += messages
        
        # Longer break between channels
        await asyncio.sleep(30)
    
    # Summary
    print(f"\n✅ Event-driven scraping complete!")
    print(f"   Total messages: {total_messages:,}")
    print(f"   Average per channel: {total_messages/len(all_channels):,.0f}")
    
    # Show channel statistics
    print(f"\n📊 Top channels by message count:")
    sorted_stats = sorted(channel_stats.items(), key=lambda x: x[1], reverse=True)
    for channel, count in sorted_stats[:10]:
        print(f"   {channel}: {count:,} messages")
    
    conn.close()
    await client.disconnect()

### Time and Data Estimates

In [ ]:
def estimate_time_and_data():
    """
    Estimate time and data volume for event-driven approach
    """
    
    # Assumptions
    channels = 24
    days_baseline = len(sampling_dates)
    avg_messages_per_channel_per_day = 50
    
    # Event-driven approach
    total_messages_sampled = channels * days_baseline * avg_messages_per_channel_per_day
    
    # Original approach
    total_days_full = (END_DATE - START_DATE).days
    total_messages_full = channels * total_days_full * avg_messages_per_channel_per_day
    
    # Time estimates (with rate limiting)
    seconds_per_message = 0.1  # Faster with sampling
    time_sampled_hours = (total_messages_sampled * seconds_per_message) / 3600
    time_full_hours = (total_messages_full * seconds_per_message) / 3600
    
    print("📊 Comparison: Event-Driven vs Full Scraping")
    print(f"\nFull Scraping:")
    print(f"  Messages: {total_messages_full:,}")
    print(f"  Time: {time_full_hours:.1f} hours ({time_full_hours/24:.1f} days)")
    
    print(f"\nEvent-Driven Sampling:")
    print(f"  Messages: {total_messages_sampled:,} ({total_messages_sampled/total_messages_full*100:.1f}% of full)")
    print(f"  Time: {time_sampled_hours:.1f} hours ({time_sampled_hours/24:.1f} days)")
    print(f"  Time saved: {100 - time_sampled_hours/time_full_hours*100:.1f}%")
    
    print(f"\n✨ Key Benefits:")
    print(f"  - Captures all major escalation periods")
    print(f"  - Maintains baseline coverage throughout conflict")
    print(f"  - Reduces scraping time by ~70%")
    print(f"  - Focuses on high-priority channels during critical periods")

estimate_time_and_data()

### Database and Helper Functions

In [ ]:
def create_minimal_database(db_path='telegram_sampled_data.db'):
    """Minimal database for AI processing"""
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Simplified messages table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS messages_minimal (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            channel_username TEXT,
            channel_category TEXT,
            message_id INTEGER,
            date DATETIME,
            message_text TEXT,
            views INTEGER,
            forwards INTEGER,
            replies_count INTEGER,
            has_media BOOLEAN,
            days_from_event INTEGER,
            nearest_event TEXT,
            near_major_event BOOLEAN,
            escalation_score INTEGER,  -- To be filled by AI
            UNIQUE(channel_username, message_id)
        )
    ''')
    
    # Channel metadata
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS channels (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE,
            title TEXT,
            category TEXT,
            priority_score INTEGER,
            last_scraped DATETIME
        )
    ''')
    
    # Create indices
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_date ON messages_minimal(date)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_views ON messages_minimal(views)')
    
    conn.commit()
    return conn

# %% [markdown]
# ## 4. Text Processing Functions

# %%
def detect_language_safe(text):
    """Safely detect language with fallback"""
    if not text or len(text.strip()) < 3:
        return 'unknown'
    
    try:
        # Remove URLs and mentions for better detection
        clean_text = re.sub(r'http\\S+|@\\S+', '', text)
        if len(clean_text.strip()) > 3:
            return detect(clean_text)
    except:
        pass
    
    # Fallback: check for Cyrillic vs Latin
    cyrillic = len(re.findall(r'[а-яА-ЯёЁїЇіІєЄґҐ]', text))
    latin = len(re.findall(r'[a-zA-Z]', text))
    
    if cyrillic > latin * 2:
        return 'ru'  # Could be Russian or Ukrainian
    elif latin > cyrillic * 2:
        return 'en'
    else:
        return 'mixed'

def extract_multilingual_text(text):
    """Extract text by script/language"""
    if not text:
        return {'original': '', 'english': '', 'russian': '', 'ukrainian': ''}
    
    # Clean the text first
    text = text.strip()
    
    # Regex patterns for different scripts
    english_pattern = re.compile(r'[a-zA-Z0-9\\s\\.\\,\\!\\?\\-]+')
    cyrillic_pattern = re.compile(r'[а-яА-ЯёЁїЇіІєЄґҐ\\s\\.\\,\\!\\?\\-]+')
    
    english_text = ' '.join(english_pattern.findall(text))
    cyrillic_text = ' '.join(cyrillic_pattern.findall(text))
    
    # Try to distinguish Russian from Ukrainian (simplified)
    ukrainian_chars = set('їієґ')
    has_ukrainian = any(char in text.lower() for char in ukrainian_chars)
    
    return {
        'original': text,
        'english': english_text.strip(),
        'russian': cyrillic_text.strip() if not has_ukrainian else '',
        'ukrainian': cyrillic_text.strip() if has_ukrainian else ''
    }

def count_escalation_keywords(text):
    """Count escalation-related keywords in text"""
    if not text:
        return 0
    
    # Escalation keywords in multiple languages
    keywords = [
        # English
        'nuclear', 'NATO', 'escalation', 'war', 'attack', 'missile', 'strike',
        'offensive', 'killed', 'destroyed', 'captured', 'liberated',
        # Russian
        'ядерный', 'НАТО', 'эскалация', 'война', 'атака', 'ракета', 'удар',
        'наступление', 'убит', 'уничтожен', 'захвачен', 'освобожден',
        # Ukrainian  
        'ядерний', 'НАТО', 'ескалація', 'війна', 'атака', 'ракета', 'удар',
        'наступ', 'вбито', 'знищено', 'захоплено', 'звільнено'
    ]
    
    text_lower = text.lower()
    return sum(1 for keyword in keywords if keyword.lower() in text_lower)

def export_to_csv(conn, output_path=None):
    """
    Export scraped data to CSV with timestamp
    """
    if output_path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = OUT_DIR / f'telegram_data_{timestamp}.csv'
    
    # Query with all relevant fields
    query = '''
        SELECT 
            m.channel_username,
            m.channel_category,
            c.priority_score,
            m.message_id,
            m.date,
            m.message_text,
            m.detected_language,
            m.views,
            m.forwards,
            m.replies_count,
            m.is_forwarded,
            m.media_type,
            m.escalation_keywords,
            m.days_from_event,
            m.nearest_event,
            m.near_major_event,
            c.title as channel_title,
            c.subscribers_count
        FROM messages m
        LEFT JOIN channels c ON m.channel_username = c.username
        ORDER BY m.date DESC
    '''
    
    try:
        # Load to DataFrame
        df = pd.read_sql_query(query, conn)
        
        # Add some derived columns
        df['message_length'] = df['message_text'].str.len()
        df['has_media'] = df['media_type'].notna()
        df['engagement_score'] = df['views'].fillna(0) + (df['forwards'].fillna(0) * 10) + (df['replies_count'].fillna(0) * 5)
        
        # Save to CSV
        df.to_csv(output_path, index=False, encoding='utf-8')
        
        print(f"\n📊 Export Summary:")
        print(f"   Total messages: {len(df)}")
        if len(df) > 0:
            print(f"   Date range: {df['date'].min()} to {df['date'].max()}")
            print(f"   Channels: {df['channel_username'].nunique()}")
            print(f"   Average escalation keywords: {df['escalation_keywords'].mean():.2f}")
            print(f"   Messages near major events: {df['near_major_event'].sum()}")
        print(f"\n💾 Exported to: {output_path}")
        
        return df
        
    except Exception as e:
        print(f"❌ Error exporting to CSV: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

def find_nearest_event(date, events):
    """Find the nearest major event to a given date"""
    min_distance = float('inf')
    nearest_event = None
    
    for event in events:
        event_date = event['datetime']
        distance = abs((date - event_date).days)
        
        if distance < min_distance:
            min_distance = distance
            nearest_event = event
    
    return {
        'days_from_event': min_distance,
        'nearest_event': nearest_event['label'] if nearest_event else None,
        'near_major_event': min_distance <= 14 and nearest_event.get('major', False)
    }

# %% [markdown] 
# ## 5. Rate Limiting Implementation

# %%
class RateLimiter:
    """Sophisticated rate limiting to avoid Telegram bans"""
    
    def __init__(self):
        self.delays = {
            'message': 0.5,        # 0.5 seconds between messages
            'api_call': 2.0,       # 2 seconds between API calls
            'channel_switch': 30,  # 30 seconds between channels
            'batch': 300,          # 5 minutes between large batches
        }
        self.last_action = {}
        self.message_count = 0
        
    async def wait_if_needed(self, action_type):
        """Implement smart delays based on action type"""
        current_time = time.time()
        
        if action_type in self.last_action:
            elapsed = current_time - self.last_action[action_type]
            required_delay = self.delays.get(action_type, 1.0)
            
            # Add jitter to avoid pattern detection
            jitter = random.uniform(0.1, 0.5)
            
            if elapsed < required_delay:
                wait_time = required_delay - elapsed + jitter
                logging.debug(f"Rate limiting: waiting {wait_time:.2f}s for {action_type}")
                await asyncio.sleep(wait_time)
        
        self.last_action[action_type] = time.time()
        
        # Track messages for batch delays
        if action_type == 'message':
            self.message_count += 1
            if self.message_count % 500 == 0:
                logging.info(f"Reached {self.message_count} messages, taking extended break")
                await asyncio.sleep(self.delays['batch'])

rate_limiter = RateLimiter()

## Complete Scraping Implementation

### Scrape, TDQM, Optimized

In [ ]:
async def scrape_channel_optimized(client, channel_username, category, conn, sampling_dates, 
                                  max_messages=None, batch_size=100):
    """
    Optimized scraper with progress bars and batch processing - FIXED
    """
    
    cursor = conn.cursor()
    messages_collected = 0
    messages_buffer = []  # Buffer for batch inserts
    channel_priority = estimate_channel_importance(channel_username, category)
    
    try:
        # Get channel entity
        await rate_limiter.wait_if_needed('api_call')
        channel = await client.get_entity(channel_username)
        
        # Get subscriber count safely
        subscriber_count = getattr(channel, 'participants_count', None)
        
        # Store channel metadata
        cursor.execute('''
            INSERT OR REPLACE INTO channels 
            (username, title, category, subscribers_count, description, is_verified, priority_score, last_scraped)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            channel_username,
            getattr(channel, 'title', channel_username),
            category,
            subscriber_count,  # Can be None
            getattr(channel, 'about', ''),
            getattr(channel, 'verified', False),
            channel_priority,
            datetime.now()
        ))
        conn.commit()
        
        print(f"\n📱 Scraping {channel_username} ({category})")
        print(f"   Priority: {channel_priority}")
        print(f"   Title: {getattr(channel, 'title', 'Unknown')}")
        
        # Fix the subscriber count formatting
        if subscriber_count is not None:
            print(f"   Subscribers: {subscriber_count:,}")
        else:
            print(f"   Subscribers: Unknown (likely a channel)")
        
        # Convert sampling_dates to set for faster lookup
        sample_date_set = set(sampling_dates)
        
        # First, estimate total messages to process
        print("   Estimating message count...")
        total_in_range = 0
        messages_to_check = []
        
        # Quick scan to count messages in date range
        async for message in client.iter_messages(channel, limit=1000):  # Check last 1000
            if not message.date:
                continue
            if message.date.date() in sample_date_set:
                total_in_range += 1
                messages_to_check.append(message)
            if message.date < START_DATE:
                break
        
        print(f"   Found ~{total_in_range} messages in sampling dates (from last 1000)")
        
        # If we want to limit messages
        if max_messages and total_in_range > max_messages:
            print(f"   Limiting to {max_messages} most recent messages")
            messages_to_check = messages_to_check[:max_messages]
        
        # Process messages with progress bar
        with tqdm_sync(total=len(messages_to_check), desc=f"{channel_username}", 
                      unit="msg", leave=True, ncols=100) as pbar:
            
            for message in messages_to_check:
                # Extract text and metadata
                if message.text and len(message.text) > 10:  # Skip very short messages
                    ml_text = extract_multilingual_text(message.text)
                    lang = detect_language_safe(message.text)
                    escalation_count = count_escalation_keywords(message.text)
                    event_context = find_nearest_event(message.date, timeline_events)
                    
                    # Add to buffer instead of inserting immediately
                    messages_buffer.append((
                        channel_username,
                        category,
                        message.id,
                        message.date,
                        message.sender_id,
                        ml_text['original'][:1000],  # Limit text length
                        ml_text['english'][:500],
                        ml_text['russian'][:500],
                        ml_text['ukrainian'][:500],
                        lang,
                        getattr(message, 'views', 0),
                        getattr(message, 'forwards', 0),
                        message.replies.replies if hasattr(message, 'replies') and message.replies else 0,
                        bool(message.fwd_from),
                        str(message.fwd_from.from_id) if message.fwd_from and hasattr(message.fwd_from, 'from_id') else None,
                        type(message.media).__name__ if message.media else None,
                        bool(getattr(message, 'reactions', None)),
                        bool(message.edit_date),
                        message.edit_date,
                        escalation_count,
                        event_context['days_from_event'],
                        event_context['nearest_event'],
                        event_context['near_major_event']
                    ))
                    
                    messages_collected += 1
                    
                    # Batch insert when buffer is full
                    if len(messages_buffer) >= batch_size:
                        cursor.executemany('''
                            INSERT OR IGNORE INTO messages 
                            (channel_username, channel_category, message_id, date, sender_id,
                             message_text, message_text_english, message_text_russian, 
                             message_text_ukrainian, detected_language, views, forwards,
                             replies_count, is_forwarded, forward_from_channel, media_type,
                             has_reactions, is_edited, edit_date, escalation_keywords,
                             days_from_event, nearest_event, near_major_event)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', messages_buffer)
                        conn.commit()
                        messages_buffer = []
                        pbar.set_postfix({'saved': messages_collected, 'buffer': 0})
                    else:
                        pbar.set_postfix({'saved': messages_collected - len(messages_buffer), 
                                         'buffer': len(messages_buffer)})
                
                # Update progress bar
                pbar.update(1)
                
                # Minimal rate limiting (reduce from 0.5s to 0.1s)
                if messages_collected % 50 == 0:
                    await asyncio.sleep(0.1)
        
        # Insert remaining messages in buffer
        if messages_buffer:
            cursor.executemany('''
                INSERT OR IGNORE INTO messages 
                (channel_username, channel_category, message_id, date, sender_id,
                 message_text, message_text_english, message_text_russian, 
                 message_text_ukrainian, detected_language, views, forwards,
                 replies_count, is_forwarded, forward_from_channel, media_type,
                 has_reactions, is_edited, edit_date, escalation_keywords,
                 days_from_event, nearest_event, near_major_event)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', messages_buffer)
            conn.commit()
        
        print(f"   ✅ Collected {messages_collected} messages")
                    
    except FloodWaitError as e:
        wait_time = e.seconds + random.randint(10, 30)
        print(f"   ⚠️  Flood wait: {wait_time} seconds")
        await asyncio.sleep(wait_time)
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
    
    finally:
        conn.commit()
        
    return messages_collected

### Minimal Channel Scraper

In [ ]:
async def scrape_channel_minimal(client, channel_username, category, conn, sampling_dates, 
                                max_messages=None, batch_size=200):
    """
    Minimal scraper - just collect raw messages for AI processing
    No language detection, no keyword counting, no translation
    """
    
    cursor = conn.cursor()
    messages_collected = 0
    messages_buffer = []
    channel_priority = estimate_channel_importance(channel_username, category)
    
    try:
        # Get channel entity
        channel = await client.get_entity(channel_username)
        
        # Store channel metadata
        cursor.execute('''
            INSERT OR REPLACE INTO channels 
            (username, title, category, priority_score, last_scraped)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            channel_username,
            getattr(channel, 'title', channel_username),
            category,
            channel_priority,
            datetime.now()
        ))
        conn.commit()
        
        print(f"\n📱 Scraping {channel_username} ({category})")
        print(f"   Title: {getattr(channel, 'title', 'Unknown')}")
        
        # Convert sampling_dates to set
        sample_date_set = set(sampling_dates)
        
        # Collect messages
        print("   Collecting messages...", end='', flush=True)
        messages_to_process = []
        
        async for message in client.iter_messages(channel, limit=3000):
            if not message.date:
                continue
            if message.date.date() in sample_date_set:
                messages_to_process.append(message)
                if len(messages_to_process) % 100 == 0:
                    print(f"\r   Collecting messages... {len(messages_to_process)} found", end='', flush=True)
            if message.date < START_DATE:
                break
                
        print(f"\r   Found {len(messages_to_process)} messages in date range")
        
        # Limit if requested
        if max_messages and len(messages_to_process) > max_messages:
            print(f"   Limiting to {max_messages} messages")
            messages_to_process = messages_to_process[:max_messages]
        
        # Process messages quickly
        print("   Processing messages...", end='', flush=True)
        
        for i, message in enumerate(messages_to_process):
            if message.text and len(message.text) > 20:  # Skip very short
                
                # Find nearest event for context
                event_context = find_nearest_event(message.date, timeline_events)
                
                # Simple buffer append - no text processing
                messages_buffer.append((
                    channel_username,
                    category,
                    message.id,
                    message.date,
                    message.text[:2000],  # Just limit length
                    getattr(message, 'views', 0),
                    getattr(message, 'forwards', 0),
                    message.replies.replies if hasattr(message, 'replies') and message.replies else 0,
                    bool(message.media),
                    event_context['days_from_event'],
                    event_context['nearest_event'],
                    event_context['near_major_event']
                ))
                
                messages_collected += 1
                
                # Progress update
                if (i + 1) % 50 == 0:
                    print(f"\r   Processing messages... {i+1}/{len(messages_to_process)} saved", 
                          end='', flush=True)
                
                # Batch insert
                if len(messages_buffer) >= batch_size:
                    cursor.executemany('''
                        INSERT OR IGNORE INTO messages_minimal 
                        (channel_username, channel_category, message_id, date,
                         message_text, views, forwards, replies_count, has_media,
                         days_from_event, nearest_event, near_major_event)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', messages_buffer)
                    conn.commit()
                    messages_buffer = []
        
        # Insert remaining
        if messages_buffer:
            cursor.executemany('''
                INSERT OR IGNORE INTO messages_minimal 
                (channel_username, channel_category, message_id, date,
                 message_text, views, forwards, replies_count, has_media,
                 days_from_event, nearest_event, near_major_event)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', messages_buffer)
            conn.commit()
        
        print(f"\n   ✅ Saved {messages_collected} messages")
                    
    except Exception as e:
        print(f"\n   ❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
    
    return messages_collected

### TDQM Import

In [ ]:
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
import nest_asyncio
nest_asyncio.apply()  # Allows nested event loops in Jupyter

### Smart Sampling Strategy - Top Messages Per Day

In [ ]:
async def scrape_top_messages_per_day_with_progress(client, channel_username, category, conn, 
                                                   messages_per_day=20):
    """
    Collect only top N most viewed/engaged messages per day with progress bar
    """
    
    cursor = conn.cursor()
    messages_collected = 0
    channel_priority = estimate_channel_importance(channel_username, category)
    
    try:
        channel = await client.get_entity(channel_username)
        
        print(f"\n📱 Scanning {channel_username} for top messages...")
        print(f"   Strategy: Top {messages_per_day} messages per day")
        
        # Store channel metadata
        cursor.execute('''
            INSERT OR REPLACE INTO channels 
            (username, title, category, priority_score, last_scraped)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            channel_username,
            getattr(channel, 'title', channel_username),
            category,
            channel_priority,
            datetime.now()
        ))
        
        # First pass: collect messages with progress bar
        daily_messages = {}
        messages_scanned = 0
        
        # Create progress bar for scanning
        with tqdm(desc=f"Scanning {channel_username}", unit="msg", leave=False) as pbar:
            async for message in client.iter_messages(channel, limit=None):
                if not message.date or not message.text or len(message.text) < 20:
                    continue
                    
                # Skip if before war start
                if message.date < START_DATE:
                    break
                    
                date_key = message.date.date()
                
                if date_key not in daily_messages:
                    daily_messages[date_key] = []
                
                # Calculate engagement score
                views = getattr(message, 'views', 0) or 0
                forwards = getattr(message, 'forwards', 0) or 0
                replies = message.replies.replies if hasattr(message, 'replies') and message.replies else 0

                engagement = views + (forwards * 10) + (replies * 5)
                
                daily_messages[date_key].append((engagement, message))
                
                # Keep only top N per day
                if len(daily_messages[date_key]) > messages_per_day:
                    daily_messages[date_key].sort(key=lambda x: x[0], reverse=True)
                    daily_messages[date_key] = daily_messages[date_key][:messages_per_day]
                
                messages_scanned += 1
                pbar.update(1)
                
                # Update description with progress
                if messages_scanned % 100 == 0:
                    pbar.set_description(f"{channel_username} - {len(daily_messages)} days")
        
        print(f"   Scanned {messages_scanned} messages from {len(daily_messages)} days")
        
        # Second pass: save top messages with progress bar
        messages_buffer = []
        total_to_save = sum(len(msgs) for msgs in daily_messages.values())
        
        with tqdm(total=total_to_save, desc=f"Saving top messages", unit="msg", leave=False) as pbar:
            for date_key in sorted(daily_messages.keys()):
                for engagement, message in daily_messages[date_key]:
                    event_context = find_nearest_event(message.date, timeline_events)
                    
                    messages_buffer.append((
                        channel_username,
                        category,
                        message.id,
                        message.date,
                        message.text[:2000],
                        getattr(message, 'views', 0),
                        getattr(message, 'forwards', 0),
                        message.replies.replies if hasattr(message, 'replies') and message.replies else 0,
                        bool(message.media),
                        event_context['days_from_event'],
                        event_context['nearest_event'],
                        event_context['near_major_event']
                    ))
                    
                    messages_collected += 1
                    pbar.update(1)
                    
                    if len(messages_buffer) >= 200:
                        cursor.executemany('''
                            INSERT OR IGNORE INTO messages_minimal 
                            (channel_username, channel_category, message_id, date,
                             message_text, views, forwards, replies_count, has_media,
                             days_from_event, nearest_event, near_major_event)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', messages_buffer)
                        conn.commit()
                        messages_buffer = []
                        pbar.set_postfix({'saved': messages_collected})
        
        # Insert remaining
        if messages_buffer:
            cursor.executemany('''
                INSERT OR IGNORE INTO messages_minimal 
                (channel_username, channel_category, message_id, date,
                 message_text, views, forwards, replies_count, has_media,
                 days_from_event, nearest_event, near_major_event)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', messages_buffer)
            conn.commit()
        
        print(f"   ✅ Saved {messages_collected} top messages")
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
    
    return messages_collected


### Simple Test

In [ ]:
async def test_simple(days_back=3, max_messages_per_channel=100):
    """
    Simple test without complex progress bars
    """
    print(f"🚀 Running simple test scraper")
    print(f"   Days back: {days_back}")
    print(f"   Max messages per channel: {max_messages_per_channel}")
    
    # Test dates
    test_dates = [
        datetime.now(timezone.utc).date() - timedelta(days=i) 
        for i in range(days_back)
    ]
    print(f"   Dates: {test_dates[-1]} to {test_dates[0]}")
    
    # Test channels - let's test with just 2 first
    test_channels = [
        ('@operativnoZSU', 'pro_ukrainian'),
        ('@rybar', 'pro_russian')
    ]
    
    # Initialize client
    client = TelegramClient('telegram_simple_test', API_ID, API_HASH)
    
    try:
        print("\n📱 Connecting to Telegram...")
        await client.start(PHONE)
        print("✅ Connected!")
        
        # Create test database
        test_db = OUT_DIR / f'telegram_test_{datetime.now().strftime("%Y%m%d_%H%M%S")}.db'
        conn = create_database(test_db)
        print(f"✅ Created database: {test_db.name}")
        
        # Process channels
        total_messages = 0
        start_time = time.time()
        
        for i, (channel, category) in enumerate(test_channels, 1):
            print(f"\n{'='*60}")
            print(f"Channel {i}/{len(test_channels)}")
            
            channel_start = time.time()
            messages = await scrape_channel_simple(
                client, channel, category, conn, test_dates,
                max_messages=max_messages_per_channel,
                batch_size=50
            )
            channel_time = time.time() - channel_start
            
            total_messages += messages
            if messages > 0:
                print(f"   ⏱️  Time: {channel_time:.1f}s ({messages/channel_time:.1f} msg/s)")
            
            # Break between channels
            if i < len(test_channels):
                print(f"\n⏸️  Waiting 3 seconds...")
                await asyncio.sleep(3)
        
        # Export results
        print(f"\n{'='*60}")
        print("📊 Exporting results...")
        csv_path = OUT_DIR / f'telegram_test_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
        df = export_to_csv(conn, csv_path)
        
        # Summary
        total_time = time.time() - start_time
        print(f"\n✅ Test complete!")
        print(f"   Total messages: {total_messages}")
        print(f"   Total time: {total_time:.1f}s")
        if total_messages > 0:
            print(f"   Average speed: {total_messages/total_time:.1f} msg/s")
        print(f"   Output: {csv_path.name}")
        
        # Show sample data
        if len(df) > 0:
            print(f"\n📝 First few messages:")
            cols = ['channel_username', 'date', 'escalation_keywords', 'views']
            print(df[cols].head().to_string())
        
        conn.close()
        
    except KeyboardInterrupt:
        print("\n\n⚠️  Interrupted by user!")
        conn.close()
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        await client.disconnect()
        print("\n👋 Disconnected from Telegram")

# Run the simple test
await test_simple(days_back=3, max_messages_per_channel=100)

In [ ]:
async def quick_test_verbose(days_back=3, max_messages_per_channel=200):
    """
    Quick verbose test with progress tracking
    """
    print(f"🚀 Running quick verbose test...")
    print(f"   Days back: {days_back}")
    print(f"   Max messages per channel: {max_messages_per_channel}")
    
    # Test dates
    test_dates = [
        datetime.now(timezone.utc).date() - timedelta(days=i) 
        for i in range(days_back)
    ]
    print(f"   Dates: {test_dates[0]} to {test_dates[-1]}")
    
    # Test channels
    test_channels = [
        ('@operativnoZSU', 'pro_ukrainian'),
        ('@rybar', 'pro_russian'),
        ('@astra', 'neutral')
    ]
    
    # Initialize client
    client = TelegramClient('telegram_quick_test', API_ID, API_HASH)
    
    try:
        print("\n📱 Connecting to Telegram...")
        await client.start(PHONE)
        print("✅ Connected!")
        
        # Create test database
        test_db = OUT_DIR / f'telegram_quicktest_{datetime.now().strftime("%Y%m%d_%H%M%S")}.db'
        conn = create_database(test_db)
        print(f"✅ Created database: {test_db.name}")
        
        # Overall progress
        total_messages = 0
        start_time = time.time()
        
        for i, (channel, category) in enumerate(test_channels, 1):
            print(f"\n{'='*60}")
            print(f"Channel {i}/{len(test_channels)}")
            
            channel_start = time.time()
            messages = await scrape_channel_optimized(
                client, channel, category, conn, test_dates,
                max_messages=max_messages_per_channel,
                batch_size=50  # Smaller batches for test
            )
            channel_time = time.time() - channel_start
            
            total_messages += messages
            print(f"   ⏱️  Time: {channel_time:.1f}s ({messages/channel_time:.1f} msg/s)")
            
            # Quick break between channels
            if i < len(test_channels):
                print(f"\n⏸️  Waiting 5 seconds before next channel...")
                await asyncio.sleep(5)
        
        # Export results
        print(f"\n{'='*60}")
        print("📊 Exporting results...")
        csv_path = OUT_DIR / f'telegram_quicktest_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
        df = export_to_csv(conn, csv_path)
        
        # Summary
        total_time = time.time() - start_time
        print(f"\n✅ Test complete!")
        print(f"   Total messages: {total_messages}")
        print(f"   Total time: {total_time:.1f}s ({total_time/60:.1f} minutes)")
        print(f"   Average speed: {total_messages/total_time:.1f} msg/s")
        print(f"   Output: {csv_path}")
        
        # Show sample data
        if len(df) > 0:
            print(f"\n📝 Sample messages:")
            sample_cols = ['channel_username', 'date', 'message_text', 'escalation_keywords', 'views']
            print(df[sample_cols].head(10).to_string(max_colwidth=50))
        
        conn.close()
        
    except KeyboardInterrupt:
        print("\n\n⚠️  Interrupted by user!")
        print("Saving collected data...")
        conn.commit()
        conn.close()
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        await client.disconnect()
        print("\n👋 Disconnected from Telegram")

# Run the optimized test
await quick_test_verbose(days_back=3, max_messages_per_channel=200)

## Full War Timeline Scraping Strategy

### Scrape Full Function

In [ ]:
async def scrape_full_timeline_smart_with_progress():
    """
    Smart scraping for entire war timeline with comprehensive progress tracking
    """
    
    print("🚀 Starting smart full timeline scrape")
    print(f"   Date range: {START_DATE.date()} to {END_DATE.date()}")
    
    # Calculate expected volume
    days_of_war = (END_DATE - START_DATE).days
    total_channels = sum(len(ch) for ch in CHANNELS.values())
    avg_messages_per_day = 15
    
    estimated_total = days_of_war * total_channels * avg_messages_per_day
    print(f"   Channels: {total_channels}")
    print(f"   Estimated messages: {estimated_total:,}")
    print(f"   Target: 100-300k messages")
    
    # Initialize client
    client = TelegramClient('telegram_full_scrape', API_ID, API_HASH)
    
    try:
        await client.start(PHONE)
        print("✅ Connected to Telegram\n")
        
        # Create database
        db_path = OUT_DIR / f'telegram_full_{datetime.now().strftime("%Y%m%d_%H%M%S")}.db'
        conn = create_minimal_database(db_path)
        
        total_messages = 0
        channel_times = []
        
        # Process channels by priority
        all_channels = []
        for category, channel_list in CHANNELS.items():
            for channel in channel_list:
                priority = estimate_channel_importance(channel, category)
                all_channels.append((priority, channel, category))
        
        all_channels.sort(reverse=True, key=lambda x: x[0])
        
        # Main progress bar for channels
        with tqdm(total=len(all_channels), desc="Overall Progress", unit="channel") as main_pbar:
            
            for i, (priority, channel, category) in enumerate(all_channels, 1):
                channel_start = time.time()
                
                print(f"\n{'='*60}")
                print(f"Channel {i}/{len(all_channels)}: {channel} (Priority: {priority})")
                
                # More messages for high priority channels
                if priority >= 9:
                    messages_per_day = 20
                elif priority >= 7:
                    messages_per_day = 15
                else:
                    messages_per_day = 10
                
                messages = await scrape_top_messages_per_day_with_progress(
                    client, channel, category, conn, 
                    messages_per_day=messages_per_day
                )
                
                total_messages += messages
                channel_time = time.time() - channel_start
                channel_times.append(channel_time)
                
                # Update main progress bar
                main_pbar.update(1)
                avg_time = sum(channel_times) / len(channel_times)
                remaining_channels = len(all_channels) - i
                eta_seconds = avg_time * remaining_channels
                
                main_pbar.set_postfix({
                    'Total': f"{total_messages:,}",
                    'ETA': f"{eta_seconds/60:.0f}m"
                })
                
                # Break between channels
                await asyncio.sleep(5)
                
                # Extended break every 5 channels
                if i % 5 == 0 and i < len(all_channels):
                    print("\n⏸️  Taking 1-minute break...")
                    for _ in tqdm(range(60), desc="Break", unit="s", leave=False):
                        await asyncio.sleep(1)
        
        # Export final dataset with progress
        print(f"\n{'='*60}")
        print("📊 Exporting final dataset...")
        
        query = '''
            SELECT * FROM messages_minimal 
            ORDER BY date DESC
        '''
        
        # Show export progress
        with tqdm(desc="Exporting to CSV", unit="rows") as export_pbar:
            df = pd.read_sql_query(query, conn)
            export_pbar.update(len(df))
            
            csv_path = OUT_DIR / f'telegram_full_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            df.to_csv(csv_path, index=False)
        
        # Final summary
        total_time = sum(channel_times)
        print(f"\n✅ Scraping complete!")
        print(f"   Total messages: {total_messages:,}")
        print(f"   Total time: {total_time/60:.1f} minutes ({total_time/3600:.1f} hours)")
        print(f"   Average speed: {total_messages/total_time:.1f} msg/s")
        print(f"   Output: {csv_path}")
        
        # Channel statistics
        print(f"\n📊 Channel Statistics:")
        stats_query = '''
            SELECT channel_username, COUNT(*) as msg_count 
            FROM messages_minimal 
            GROUP BY channel_username 
            ORDER BY msg_count DESC
            LIMIT 10
        '''
        stats_df = pd.read_sql_query(stats_query, conn)
        print(stats_df.to_string(index=False))
        
        conn.close()
        
    except KeyboardInterrupt:
        print("\n\n⚠️  Scraping interrupted by user!")
        print(f"   Collected {total_messages:,} messages before interruption")
        conn.close()
        
    except Exception as e:
        print(f"\n❌ Fatal error: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        await client.disconnect()
        print("\n👋 Disconnected from Telegram")

### Header

In [ ]:
async def test_tqdm_scraper(num_channels=3):
    """Quick test of the TQDM implementation"""
    
    print("🧪 Testing TQDM scraper with limited channels")
    
    # Use only first N channels for test
    test_channels = []
    for category, channels in CHANNELS.items():
        for channel in channels[:1]:  # One from each category
            test_channels.append((channel, category))
        if len(test_channels) >= num_channels:
            break
    
    client = TelegramClient('telegram_tqdm_test', API_ID, API_HASH)
    
    try:
        await client.start(PHONE)
        print("✅ Connected\n")
        
        # Create test database
        db_path = OUT_DIR / f'telegram_tqdm_test_{datetime.now().strftime("%Y%m%d_%H%M%S")}.db'
        conn = create_minimal_database(db_path)
        
        total_messages = 0
        
        with tqdm(total=len(test_channels), desc="Test Progress", unit="channel") as pbar:
            for channel, category in test_channels:
                messages = await scrape_top_messages_per_day_with_progress(
                    client, channel, category, conn, 
                    messages_per_day=10
                )
                total_messages += messages
                pbar.update(1)
                pbar.set_postfix({'Total': total_messages})
        
        print(f"\n✅ Test complete! Collected {total_messages} messages")
        conn.close()
        
    finally:
        await client.disconnect()

# Run the test
await test_tqdm_scraper(num_channels=2)

# %% [markdown]
# ## Usage Instructions

# Run the full smart scrape:
# await scrape_full_timeline_smart_with_progress()

# Or test with a few channels first:
# await test_tqdm_scraper(num_channels=3)

### RUN FULL SCRAPE

In [ ]:
# Run the full smart scrape
await scrape_full_timeline_smart_with_progress()

### Analysis of Scraped Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import numpy as np

# Load the data
csv_path = OUT_DIR / 'telegram_full_20250605_213258.csv'
df = pd.read_csv(csv_path)

# Convert date column to datetime and remove timezone for consistency
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)

# Define channel categories based on your CHANNELS dict
channel_categories = {
    # Pro-Ukrainian
    '@ssternenko': 'Pro-Ukrainian',
    '@operativnoZSU': 'Pro-Ukrainian', 
    '@voynareal': 'Pro-Ukrainian',
    '@DeepStateUA': 'Pro-Ukrainian',
    '@horevica': 'Pro-Ukrainian',
    '@ukr_sof': 'Pro-Ukrainian',
    '@aerobomber': 'Pro-Ukrainian',
    '@combat_ftg': 'Pro-Ukrainian',
    
    # Pro-Russian
    '@rybar': 'Pro-Russian',
    '@starshe_eddy': 'Pro-Russian',
    '@wargonzo': 'Pro-Russian',
    '@grey_zone': 'Pro-Russian',
    '@vysokygovorit': 'Pro-Russian',
    '@milinfolive': 'Pro-Russian',
    '@voenkorKotenok': 'Pro-Russian',
    '@romanov_92': 'Pro-Russian',
    
    # Neutral/Independent
    '@astra': 'Neutral/Independent',
    '@CITeam_en': 'Neutral/Independent',
    '@bellingcat': 'Neutral/Independent',
    '@GirkinGirkin': 'Neutral/Independent',
    
    # Official
    '@V_Zelenskiy_official': 'Official',
    '@spravdi': 'Official',
    '@mod_russia': 'Official',
    '@MID_RF': 'Official'
}

# Add category to dataframe
df['category'] = df['channel_username'].map(channel_categories)

# Create figure with subplots
fig = plt.figure(figsize=(20, 16))

# 1. Messages over time by category
ax1 = plt.subplot(3, 2, 1)
df_daily = df.groupby([pd.Grouper(key='date', freq='D'), 'category']).size().reset_index(name='count')
pivot_daily = df_daily.pivot(index='date', columns='category', values='count').fillna(0)

# Stacked area chart
colors = {'Pro-Ukrainian': '#0057B7', 'Pro-Russian': '#D52B1E', 
          'Neutral/Independent': '#808080', 'Official': '#FFD700'}
pivot_daily.plot(kind='area', stacked=True, ax=ax1, color=[colors.get(col, '#333') for col in pivot_daily.columns])
ax1.set_title('Messages Over Time by Category (Daily)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Number of Messages')
ax1.legend(title='Category', loc='upper left')

# Add major events
events = [
    ('2022-02-24', 'Invasion'),
    ('2022-09-11', 'Kharkiv Liberation'),
    ('2023-06-06', 'Dam Destroyed'),
    ('2024-08-06', 'Kursk Incursion'),
    ('2024-11-05', 'Trump Elected')
]
for date, label in events:
    event_date = pd.to_datetime(date)
    if pivot_daily.index.min() <= event_date <= pivot_daily.index.max():
        ax1.axvline(x=event_date, color='red', linestyle='--', alpha=0.5)
        ax1.text(event_date, ax1.get_ylim()[1]*0.9, label, rotation=90, fontsize=8)

# 2. Total messages by channel
ax2 = plt.subplot(3, 2, 2)
channel_counts = df['channel_username'].value_counts().head(15)
channel_counts_df = pd.DataFrame({'channel': channel_counts.index, 'count': channel_counts.values})
channel_counts_df['category'] = channel_counts_df['channel'].map(channel_categories)

# Color bars by category
colors_list = [colors.get(cat, '#333') for cat in channel_counts_df['category']]
bars = ax2.barh(channel_counts_df['channel'], channel_counts_df['count'], color=colors_list)
ax2.set_xlabel('Number of Messages')
ax2.set_title('Top 15 Channels by Message Count', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# Add value labels
for bar in bars:
    width = bar.get_width()
    ax2.text(width, bar.get_y() + bar.get_height()/2, f'{int(width):,}', 
             ha='left', va='center', fontsize=8)

# 3. Category distribution pie chart
ax3 = plt.subplot(3, 2, 3)
category_counts = df['category'].value_counts()
wedges, texts, autotexts = ax3.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%',
        colors=[colors.get(cat, '#333') for cat in category_counts.index],
        explode=[0.05 if cat == 'Pro-Ukrainian' else 0 for cat in category_counts.index])
ax3.set_title('Message Distribution by Category', fontsize=14, fontweight='bold')

# Add total counts to labels
for i, (text, autotext) in enumerate(zip(texts, autotexts)):
    text.set_text(f"{text.get_text()}\n({category_counts.values[i]:,})")
    autotext.set_fontsize(10)
    autotext.set_fontweight('bold')

# 4. Weekly message volume heatmap
ax4 = plt.subplot(3, 2, 4)
df['week'] = df['date'].dt.to_period('W')
df['dayofweek'] = df['date'].dt.day_name()
weekly_heatmap = df.groupby(['week', 'dayofweek']).size().reset_index(name='count')

# Limit to last 20 weeks for readability
recent_weeks = sorted(df['week'].unique())[-20:]
weekly_heatmap = weekly_heatmap[weekly_heatmap['week'].isin(recent_weeks)]

# Pivot for heatmap
pivot_heatmap = weekly_heatmap.pivot(index='week', columns='dayofweek', values='count').fillna(0)
# Reorder days
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
pivot_heatmap = pivot_heatmap.reindex(columns=days_order)

sns.heatmap(pivot_heatmap.T, cmap='YlOrRd', ax=ax4, cbar_kws={'label': 'Messages'})
ax4.set_title('Message Activity Heatmap (Last 20 Weeks)', fontsize=14, fontweight='bold')
ax4.set_xlabel('Week')
ax4.set_ylabel('Day of Week')

# 5. Engagement metrics by category
ax5 = plt.subplot(3, 2, 5)
# Calculate average engagement per category
df['engagement_score'] = df['views'].fillna(0) + (df['forwards'].fillna(0) * 10) + (df['replies_count'].fillna(0) * 5)
engagement_by_cat = df.groupby('category')['engagement_score'].agg(['mean', 'median'])

x = np.arange(len(engagement_by_cat))
width = 0.35

bars1 = ax5.bar(x - width/2, engagement_by_cat['mean'], width, label='Mean', 
                 color=[colors.get(cat, '#333') for cat in engagement_by_cat.index])
bars2 = ax5.bar(x + width/2, engagement_by_cat['median'], width, label='Median', alpha=0.7,
                 color=[colors.get(cat, '#333') for cat in engagement_by_cat.index])

ax5.set_xlabel('Category')
ax5.set_ylabel('Engagement Score')
ax5.set_title('Average Engagement by Category', fontsize=14, fontweight='bold')
ax5.set_xticks(x)
ax5.set_xticklabels(engagement_by_cat.index, rotation=45)
ax5.legend()
ax5.grid(axis='y', alpha=0.3)

# 6. Time series of Pro-Ukrainian vs Pro-Russian balance
ax6 = plt.subplot(3, 2, 6)
balance_df = df[df['category'].isin(['Pro-Ukrainian', 'Pro-Russian'])]
daily_balance = balance_df.groupby([pd.Grouper(key='date', freq='D'), 'category']).size().unstack(fill_value=0)

if 'Pro-Ukrainian' in daily_balance.columns and 'Pro-Russian' in daily_balance.columns:
    daily_balance['ratio'] = daily_balance['Pro-Ukrainian'] / (daily_balance['Pro-Russian'] + 1)  # +1 to avoid division by zero
    daily_balance['ratio_ma'] = daily_balance['ratio'].rolling(window=7).mean()
    
    ax6.plot(daily_balance.index, daily_balance['ratio_ma'], color='green', linewidth=2)
    ax6.axhline(y=1, color='black', linestyle='--', alpha=0.5)
    ax6.fill_between(daily_balance.index, 1, daily_balance['ratio_ma'], 
                     where=(daily_balance['ratio_ma'] >= 1), color='#0057B7', alpha=0.3, label='Pro-Ukrainian')
    ax6.fill_between(daily_balance.index, 1, daily_balance['ratio_ma'], 
                     where=(daily_balance['ratio_ma'] < 1), color='#D52B1E', alpha=0.3, label='Pro-Russian')
    
    ax6.set_title('Pro-Ukrainian vs Pro-Russian Message Ratio (7-day MA)', fontsize=14, fontweight='bold')
    ax6.set_xlabel('Date')
    ax6.set_ylabel('Ratio (Pro-UKR / Pro-RU)')
    ax6.set_yscale('log')
    ax6.legend()
    ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIG_DIR / 'telegram_data_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Print summary statistics
print("\n📊 TELEGRAM DATA SUMMARY")
print("=" * 60)
print(f"Total messages collected: {len(df):,}")
print(f"Date range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
print(f"Total channels: {df['channel_username'].nunique()}")
print(f"Average messages per channel: {len(df) / df['channel_username'].nunique():,.0f}")
print(f"Average messages per day: {len(df) / (df['date'].max() - df['date'].min()).days:,.0f}")

print("\n📈 CATEGORY BREAKDOWN")
print("-" * 40)
for cat, count in df['category'].value_counts().items():
    percentage = (count / len(df)) * 100
    print(f"{cat:<20} {count:>8,} ({percentage:>5.1f}%)")

print("\n🎯 TOP ENGAGEMENT POSTS")
print("-" * 40)
top_posts = df.nlargest(5, 'engagement_score')[['channel_username', 'date', 'message_text', 'views', 'forwards', 'engagement_score']]
for idx, post in top_posts.iterrows():
    print(f"\nChannel: {post['channel_username']} | Date: {post['date'].strftime('%Y-%m-%d')}")
    print(f"Views: {post['views']:,} | Forwards: {post['forwards']:,} | Score: {post['engagement_score']:,.0f}")
    print(f"Text: {post['message_text'][:100]}...")

print("\n💾 MESSAGES NEAR MAJOR EVENTS")
print("-" * 40)
near_events = df[df['near_major_event'] == True]
event_summary = near_events.groupby('nearest_event').size().sort_values(ascending=False)
for event, count in event_summary.items():
    print(f"{event:<20} {count:>8,} messages")

print("\n✅ Visualization saved to:", FIG_DIR / 'telegram_data_analysis.png')